In [1]:
from proton_decay_study.models.stages import *
from keras.layers.convolutional import MaxPooling3D, Conv3D
from keras.layers import InputLayer, Dropout, Dense, Flatten
from keras.models import Sequential
from keras import optimizers
from keras.regularizers import l1, l2
import logging


Using TensorFlow backend.


In [2]:
class MyGen(object):
    @property
    def output(self):
        return (1,3,9600,3600)
    @property
    def input(self):
        return 10
gen = MyGen()

In [8]:
class MyNet(Sequential):
    fc_name='fc1'
    
    def __init__(self, generator):
        super(MyNet, self).__init__()
        self.generator = generator
        self.assemble()
        self.sgd = optimizers.RMSprop(lr=1e-9, rho=0.999, epsilon=1e-9, decay=1e-9)
        self.compile(loss='mean_squared_error', optimizer=self.sgd,
                 metrics=['accuracy'])
    def assemble(self):
        self._input = InputLayer(input_shape=self.generator.output,
                        dtype='float32',
                        name='input_1')
        self.add(self._input)

        self.add(Conv3D(32, (1, 5, 5), strides=(1, 4, 4),
                       activation='relu', padding='same',
                       data_format='channels_first',
                       kernel_initializer='random_uniform',
                       bias_initializer='zeros',
                       kernel_regularizer=l2(0.01),
                       activity_regularizer=l1(0.01),
                       name='conv1_1'))
        self.add(MaxPooling3D((1, 4, 4), strides=(1, 4, 4),
                         data_format='channels_first',
                         name='pool_1_1'))
        
        self.add(Flatten(name='flatten_{}'.format(self.fc_name)))
        self.add(Dense(self.generator.input,
                  activation='softmax',
                  kernel_initializer='random_uniform',
                  bias_initializer='zeros',
                  kernel_regularizer=l2(0.01),
                  activity_regularizer=l1(0.01),
                  name=self.fc_name))
        
net = MyNet(gen)